In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

Using TensorFlow backend.


In [3]:
import cv2
import numpy as np
import argparse
import zipfile
from tqdm import tqdm
import os
import xmltodict

In [4]:
#read video file frame by frame, beginning and ending with a timestamp
def read_video_segments(video_path,start_ms,end_ms,i):
    vid = cv2.VideoCapture(video_path)
    fps = vid.get(cv2.CAP_PROP_FPS)
    start_frame=int((start_ms/1000.0)*fps)
    end_frame=int((end_ms/1000.0)*fps)
    vid_length=0
    with tqdm(total=end_frame-start_frame+1) as pbar: #init the progressbar,with max length of the given segment
        vid.set(cv2.CAP_PROP_POS_FRAMES,start_frame)
        while(vid.isOpened()):
            ret, frame = vid.read() # if ret is false, frame has no content
            if (vid_length+start_frame)==end_frame:
                pbar.update(1)
                break
            path='../test_frames/'
            name=str(i)+'__'+str(start_frame+vid_length)+'.png'
            cv2.imwrite(path+name,frame)
            pbar.update(1) #update the progressbar
            vid_length+=1 #increase the vid_length counter

In [5]:
def get_shots(azp):
    zip_ref = zipfile.ZipFile(azp)
    inxmlstr = zip_ref.read('content.xml')
    doc = xmltodict.parse(inxmlstr)
    shots = []
    for a in doc['package']['annotations']['annotation']:
        if a['@type'] == '#Shot':
            begin_ms = int(a['millisecond-fragment']['@begin'])
            end_ms = int(a['millisecond-fragment']['@end'])

            shots.append((begin_ms, end_ms))
    return sorted(shots)

In [143]:
# with open('test.txt','w') as f:
#     print('asf',file=f)

In [160]:
videos_path='videos/'
features_path='features_videos/'

for entry in os.listdir(features_path):
    if not os.path.isdir(features_path+entry):
        
#         print(True)
    print(entry)

True
David_Ed_Where_Do_We_Go_From_Here
True
Her_bluray_Szene 11_25fps
True
Maltsev_Sem_Occupy_Wall_Street


In [148]:
import xml.etree.ElementTree as ET
def read_shotdetect_xml(path):
    tree = ET.parse(path)
    root=tree.getroot().findall('content')
    info={}
    timestamps=[]
    for child in root[0].iter():
    #     if child.tag == 'media':
    #         for grandchild in child.iter():
    #             if grandchild.tag == 'fps':
    #                 info['fps']=float(grandchild.text)
    #             if grandchild.tag == 'height':
    #                 info['resolution']=[int(grandchild.text)]
    #             if grandchild.tag == 'width':
    #                 info['resolution'].append(int(grandchild.text))
        if child.tag == 'shot':
            items=child.items()
            timestamps.append((int(items[4][1]),int(items[4][1])+int(items[2][1])-1))
    #         break
    return timestamps#,info

In [149]:
read_shotdetect_xml('tests/shot_detect4/result.xml')

[(0, 290),
 (291, 4373),
 (4375, 9374),
 (9375, 15915),
 (15916, 20373),
 (20375, 25249),
 (25250, 31249),
 (31250, 37124),
 (37125, 40624),
 (40625, 43082),
 (43083, 45290),
 (45291, 47165),
 (47166, 49040),
 (49041, 51706),
 (51708, 58123),
 (58125, 74374),
 (74375, 79374),
 (79375, 79582),
 (79583, 89040),
 (89041, 101206),
 (101208, 103998),
 (104000, 108999),
 (109000, 111582),
 (111583, 111623),
 (111625, 114040),
 (114041, 116331),
 (116333, 118082),
 (118083, 119498),
 (119500, 121457),
 (121458, 121498),
 (121500, 124749),
 (124750, 128040),
 (128041, 133081),
 (133083, 137165),
 (137166, 137206),
 (137208, 169998),
 (170000, 174040),
 (174041, 176706),
 (176708, 179498),
 (179500, 180915),
 (180916, 182331),
 (182333, 183457),
 (183458, 185873),
 (185875, 191499)]

In [45]:
xmltodict.parse('tests/shot_detect/result.xml',encoding='ISO-8859-1')

ExpatError: not well-formed (invalid token): line 1, column 5

In [12]:
shots=get_shots('azp/CompanyMen_v1.0-split-012-Bobby_being_angry.azp')

In [13]:
# for i,(begin,end) in enumerate(shots):
#     read_video_segments('../Wells_John_CompanyMen_full.mp4',begin,end,i)

In [ ]:
min(all_distances['distance'])

In [ ]:
all_distances['distance'][np.argmin(all_distances['distance'])]

In [ ]:
NASNetMobile_model = NASNetLarge(weights='imagenet', include_top=False,pooling='max',input_shape=(331,331,3))  

In [ ]:
vgg_model = VGG16(weights='imagenet', include_top=False,pooling='max')

In [ ]:
img_path = 'comparison_4.png'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)

In [ ]:
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = vgg_model.predict(x)
print(np.shape(features))

In [ ]:
from scipy.spatial.distance import euclidean

In [ ]:
euclidean(features,features)